Ref: https://github.com/SchindlerLiang/VAE-for-Anomaly-Detection

In [1]:
import os
import random

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from evaluator import evaluate
from data_loader import load_kdd_cup_urc, load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4, load_power_demand # Univariate Datasets
from data_loader import load_nasa, load_ecg, load_gesture, load_smd # Multivariate Datasets

from tensorflow import keras
from tensorflow.keras import layers
from tqdm.notebook import tqdm

# THESE LINES ARE FOR REPRODUCIBILITY
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [4]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=1
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [5]:
def LSTM_VAE(X_train):
    latent_dim = 16

    encoder_inputs = keras.Input(shape=(X_train.shape[1], X_train.shape[2]))
    x = layers.LSTM(64, return_sequences=True)(encoder_inputs)
    x = layers.LSTM(32)(x)
    z_mean = layers.Dense(latent_dim, name="z_mean")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
    z = Sampling()([z_mean, z_log_var])
    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

    latent_inputs = keras.Input(shape=(latent_dim,))
    x = layers.RepeatVector(X_train.shape[1])(latent_inputs)
    x = layers.LSTM(32, return_sequences=True)(x)
    x = layers.LSTM(64)(x)
    x = layers.Dense(X_train.shape[1] *  X_train.shape[2])(x)
    decoder_outputs = layers.Reshape([X_train.shape[1], X_train.shape[2]])(x)
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

    model = VAE(encoder, decoder)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001))
    history = model.fit(X_train, epochs=50, batch_size=128, verbose=0)
    return model

### Yahoo S5

In [6]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [7]:
for loader in [load_yahoo_A1, load_yahoo_A2, load_yahoo_A3, load_yahoo_A4]:
    datasets = loader(64, 1)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)
    
        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/56 [00:00<?, ?it/s]

yahoo_A1 0.999999948484851 0.9848484768795222 0.9999999981470107
yahoo_A1 0.9999999416666693 0.9166666376815787 0.9999999913849764
yahoo_A1 0.9777777234567926 0.9514492572839868 0.999557629337239
yahoo_A1 0.999999930000003 0.5999999693333345 0.9999999797237573
yahoo_A1 0.9230768591716004 0.7922077522590122 0.9980158561333934
yahoo_A1 0.9230768591716004 0.7647058436672041 0.9956349038267986
yahoo_A1 0.999999948484851 0.9848484767788666 0.9999999981601732
yahoo_A1 0.983870916961501 0.9838277861871575 0.991221484289207
yahoo_A1 0.9322708656116595 0.9701074089636132 0.9877399217730798
yahoo_A1 0.9999999487179513 0.9871794802015625 0.9999999983719279
yahoo_A1 0.649999955475003 0.6814825280271881 0.709558802239739
yahoo_A1 0.9937887686431877 0.9866027870087215 0.9996704874044167
yahoo_A1 0.9999999483870992 0.9838709593548329 0.9999999980537635
yahoo_A1 0.9230768686390559 0.7221973300069839 0.9930555497499013
yahoo_A1 0.9999999487013013 0.9740259679924831 0.9999999983645984
yahoo_A1 0.9999999

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/92 [00:00<?, ?it/s]

yahoo_A3 0.25396823169564314 0.00872130222697063 0.0731309495821311
yahoo_A3 0.442477841491114 0.15411338849427517 0.2756639238497847
yahoo_A3 0.7249999504921909 0.6484591998728928 0.8486467699025526
yahoo_A3 0.25396823169564314 0.00266806519236573 0.04220159812623962
yahoo_A3 0.6314151976538852 0.0832432687580802 0.18515377867348662
yahoo_A3 0.41726615387920135 0.006303056573007273 0.04302540240865644
yahoo_A3 0.5852089616298454 0.19395608566245498 0.1849967069643177
yahoo_A3 0.45614031550631234 0.07701259018435355 0.17775349453678133
yahoo_A3 0.5210083646578659 0.017765224650788395 0.14423502063428945
yahoo_A3 0.7134502462945892 0.4312674328519862 0.41706829109124843
yahoo_A3 0.637770854195864 0.21691656961536682 0.25639030680153013
yahoo_A3 0.5783521396394755 0.4634619675981355 0.46970549043255505
yahoo_A3 0.19301846297450495 0.00142045541775027 0.10941475456995176
yahoo_A3 0.6766916843462069 0.02697169209311013 0.032604072907543066
yahoo_A3 0.11563168070466742 0.002571192338179495 

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

yahoo_A4 0.4853700147102329 0.3399696426036133 0.573594045239534
yahoo_A4 0.2544731387231303 0.01901853524344787 0.12147789490706809
yahoo_A4 0.41726615387920135 0.006344003765696372 0.10679045566215023
yahoo_A4 0.2945736181779962 0.03952241846650187 0.1972813760890338
yahoo_A4 0.7228915165553815 0.81232225385025 0.8052707348018777
yahoo_A4 0.7741934993743688 0.8500618710372089 0.8883061459251756
yahoo_A4 0.47887318904098036 0.48156165407522905 0.6356954289518165
yahoo_A4 0.3673469086517006 0.170447019733724 0.46949684383476864
yahoo_A4 0.5942491593197877 0.24501287095184457 0.318006279858834
yahoo_A4 0.45070419027970904 0.0069178572014325 0.04288103965973015
yahoo_A4 0.637770854195864 0.3286424849875839 0.3758882345990707
yahoo_A4 0.3882783568463308 0.005563924512424259 0.06608312302608893
yahoo_A4 0.45070419027970904 0.0015974454027983312 -0.0
yahoo_A4 0.3551401575753364 0.13028095441711915 0.3443509510055781
yahoo_A4 0.07439824225732532 0.0011792459581944472 -0.0
yahoo_A4 0.25396823

In [8]:
yahoo_results = pd.DataFrame(total_scores)
yahoo_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
yahoo_A1,0.741957,0.591924,0.706386
yahoo_A2,0.576314,0.386187,0.679169
yahoo_A3,0.527801,0.230681,0.331648
yahoo_A4,0.505855,0.224112,0.315003


### NASA

In [9]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [10]:
for loader in [load_nasa]:
    datasets = loader(100, 100)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)
        
        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

D1 0.25300017462469454 0.1310774099523998 0.35531824696954745
D2 0.2699530282282525 0.5840009300457375 0.5173913038147389


In [11]:
nasa_results = pd.DataFrame(total_scores)
nasa_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.253000,0.131077,0.355318
D2,0.269953,0.584001,0.517391


### SMD

In [12]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [13]:
for loader in [load_smd]:
    datasets = loader(64, 1)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        tf.keras.backend.clear_session()

        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])   

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

smd 0.20232181144029696 0.5556857798256385 0.521888680406877
smd 0.7080223384287494 0.551710535514189 0.9191231912768012
smd 0.24475759898047408 0.219950937840305 0.5903531431563934
smd 0.26386073698048373 0.2890652644733706 0.5956179787098114
smd 0.689655126355723 0.7416842166394192 0.9499106269538564
smd 0.726425765802517 0.8022758852529401 0.8847852533178482
smd 0.38105170869074706 0.3943878784882371 0.6394658963354369
smd 0.2918424327107888 0.29273626205822867 0.7250530178185837
smd 0.5189747196290893 0.5763596055986044 0.8654698589516957
smd 0.2596273980668859 0.30899521959368986 0.5436930778435464
smd 0.7339955349140422 0.7779197082812668 0.8677092922942592
smd 0.24600048948742329 0.26278667750039353 0.5396083039471353
smd 0.5496182710351756 0.5249348268195897 0.8358061982424857
smd 0.5189553099967452 0.5240781476950535 0.7110228143980261
smd 0.5284749841687082 0.5886110064415466 0.9159807520769212
smd 0.9677418850899386 0.6730995648261773 0.9809677285777375
smd 0.765308624621453

In [14]:
smd_results = pd.DataFrame(total_scores)
smd_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
smd,0.509571,0.501879,0.764279


### ECG

In [15]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [16]:
for loader in [load_ecg]:
    datasets = loader(32, 16)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(f'D{i+1}')
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(f'D{i+1}', np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

D1 0.6206896038049976 0.5326525936008679 0.6750068037272775
D2 0.363636315151521 0.23541899528696025 0.5590456747449235
D3 0.11320751384835404 0.04367631207943112 0.44307798338600746
D4 0.41666661458333903 0.2013686028274416 0.6715630806541828
D5 0.3999999612839539 0.16615675062595867 0.67033798814393
D6 0.31249995117188206 0.15095157139101562 0.6854580546040853
D7 0.04918032266275918 0.01636272071549821 0.405622129457916
D8 0.1951219282702121 0.09269561477001245 0.4519237190097242
D9 0.38805966963689265 0.17316629517967963 0.37851313122196206


In [17]:
ecg_results = pd.DataFrame(total_scores)
ecg_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
D1,0.620690,0.532653,0.675007
D2,0.363636,0.235419,0.559046
D3,0.113208,0.043676,0.443078
D4,0.416667,0.201369,0.671563
D5,0.400000,0.166157,0.670338
D6,0.312500,0.150952,0.685458
D7,0.049180,0.016363,0.405622
D8,0.195122,0.092696,0.451924
D9,0.388060,0.173166,0.378513


### Power Demand

In [18]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [19]:
for loader in [load_power_demand]:
    datasets = loader(64, 1)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]
        
        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc']) 

  0%|          | 0/1 [00:00<?, ?it/s]

power_demand 0.3606206231571944 0.10335052401683913 0.1360581420800021


In [20]:
power_results = pd.DataFrame(total_scores)
power_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
power_demand,0.360621,0.103351,0.136058


### 2D Gesture

In [21]:
total_scores = {'dataset': [], 'f1': [], 'pr_auc': [], 'roc_auc': []}

In [22]:
for loader in [load_gesture]:
    datasets = loader(64, 1)
    x_trains, x_tests, y_tests = datasets['x_train'], datasets['x_test'], datasets['y_test']
    
    for i in tqdm(range(len(x_trains))):
        X_train = x_trains[i]
        X_test = x_tests[i]

        model = LSTM_VAE(X_train)
        
        X_test_rec = model.decoder.predict(model.encoder.predict(X_test)[-1])
        scores = evaluate(X_test, X_test_rec, y_tests[i], is_reconstructed=True)

        total_scores['dataset'].append(loader.__name__.replace('load_', ''))
        total_scores['f1'].append(np.max(scores['f1']))
        total_scores['pr_auc'].append(scores['pr_auc'])
        total_scores['roc_auc'].append(scores['roc_auc'])
        print(loader.__name__.replace('load_', ''), np.max(scores['f1']), scores['pr_auc'], scores['roc_auc'])  

  0%|          | 0/1 [00:00<?, ?it/s]

gesture 0.4203232924120907 0.19432085097920657 0.33302161234566896


In [23]:
gesture_results = pd.DataFrame(total_scores)
gesture_results.groupby('dataset').mean()

,f1,pr_auc,roc_auc
dataset,,,
gesture,0.420323,0.194321,0.333022
